In [6]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()
convert("train-images.idx3-ubyte", "train-labels.idx1-ubyte",
        "mnist_train.csv", 60000)
convert("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte",
        "mnist_test.csv", 10000)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
data_test=pd.read_csv("mnist_test.csv")

In [9]:
data_test.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data_train=pd.read_csv("mnist_train.csv")
data_train.head()

,5,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
m,n=data_train.shape
print(m,n)

59999 785


converting rows to columns

In [111]:
data_train_array=np.array(data_train)
data_train_array=data_train_array.T
data_train_array.shape


(785, 59999)

In [113]:
data_test_array=np.array(data_test)
data_test_array=data_test_array.T
data_test_array.shape

(785, 9999)

In [171]:
y_train=data_train_array[0]
x_train=data_train_array[1:n]
x_train=x_train/255
y_test=data_test_array[0]
x_test=data_test_array[1:n]


In [140]:
x_train[:,0].shape

(784,)

In [142]:
x_train.shape

(784, 59999)

In [144]:
y_train.size

59999

In [146]:
y_train.max(0)

9

### Neural Network

In [1]:
def init_parameters():
    w1=np.random.rand(10, 784)-0.5
    b1=np.random.rand(10, 1)-0.5
    w2=np.random.rand(10, 10)-0.5
    b2=np.random.rand(10, 1)-0.5
    return w1,w2,b1,b2

def ReLU(x):
    return np.maximum(x,0)

def softmax(x):
   
    exp_x = np.exp(x) 
    return exp_x / np.sum(exp_x)

def forward_prop(w1,w2,b1,b2,x):
    z1 = np.dot(w1, x) + b1
    A1 = ReLU(z1)
    
    z2 = np.dot(w2, A1) + b2
    A2 = softmax(z2)
   
    return z1, z2, A1, A2


def one_hot_encode(y):
    one_hot_y=np.zeros((y.size,(y.max(0)+1)))
   
    i=np.arange(y.size)
    one_hot_y[i,y]=1
    one_hot_y=one_hot_y.T
    return one_hot_y

def der_relu(z):
   return z>0
    
def back_prop(z1,z2,A1,A2,w2,x,y):
    m = y.size
    one_hot_y = one_hot_encode(y)
    dz2 = A2 - one_hot_y
    
    dw2 = np.dot(dz2, A1.T) / m
    db2 = np.sum(dz2) / m
    dz1 = np.dot(w2.T, dz2) * der_relu(z1)
    dw1 = np.dot(dz1, x.T) / m
    db1 = np.sum(dz1) / m
    return dw1,dw2,db1,db2

def update_param(w1,w2,b1,b2,dw1,dw2,db2,db1,a):
    w1=w1-a*dw1
    w2=w2-a*dw2
    b1=b1-a*db1
    b2=b2-a*db2
    return w1,w2,b1,b2

In [136]:
def get_predict(A2):
    return np.argmax(A2,0)

def accuracy(pred,y):
    print(pred,y)
    return np.sum(pred==y)/y.size

def gradient_decend(x,y,epochs,lr):
    w1,w2,b1,b2=init_parameters()
    for i in range (epochs):
        z1,z2,A1,A2=forward_prop(w1,w2,b1,b2,x)
        dw1,dw2,db1,db2=back_prop(z1,z2,A1,A2,w2,x,y)
        w1,w2,b1,b2=update_param(w1,w2,b1,b2,dw1,dw2,db2,db1,lr)
        if(i%50==0):
            print("epochs=",i)
            print("accuracy=",accuracy(get_predict(A2),y))
    return w1,w2,b1,b2
    

In [151]:
w1,w2,b1,b2=gradient_decend(x_train,y_train,2000,0.1)


epochs= 0
[0 9 9 ... 0 0 0] [0 4 1 ... 5 6 8]
accuracy= 0.09875164586076435
epochs= 50
[0 5 6 ... 0 0 6] [0 4 1 ... 5 6 8]
accuracy= 0.34745579092984885
epochs= 100
[0 5 1 ... 0 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.6268437807296788
epochs= 150
[0 5 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.7116951949199153
epochs= 200
[0 5 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.7592959882664712
epochs= 250
[0 3 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.790729845497425
epochs= 300
[0 3 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8108135135585594
epochs= 350
[0 4 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8253804230070502
epochs= 400
[0 4 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8354139235653928
epochs= 450
[0 4 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8435140585676428
epochs= 500
[0 4 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8495641594026567
epochs= 550
[0 4 1 ... 5 0 8] [0 4 1 ... 5 6 8]
accuracy= 0.8547142452374207
epochs= 600
[0 4 1 ... 5 6 8] [0 4 1 ... 5 6 8]
accuracy= 0.8590143169052817
e

In [187]:
def pred(x,w1,w2,b1,b2):
    z1,z2,A1,A2=forward_prop(w1,w2,b1,b2,x)
    return np.argmax(A2,0)




In [189]:
predic=pred(x_test,w1,w2,b1,b2)
accuracy(predic,y_test)

[2 1 0 ... 4 5 6] [2 1 0 ... 4 5 6]


0.8921892189218922